# Import Libraries and Clone Github

In [1]:
# Import Libraries
import os
import pandas as pd
from google.colab import userdata
import google.generativeai as genai

In [2]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

fatal: destination path 'Financial_Sentiment_LLM' already exists and is not an empty directory.


# Import Data

### NOTE: Updated text processing file to add unique identifier, and also concatenate the source to the identifier.  Make sure to separate the earnings presentations from the earnings calls also

In [3]:
# Import all cleaned data files
invest_df1 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/SEC_Filings.csv')

# Merge into single df
text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
display(text_df.shape)
display(text_df.head())
display(text_df.tail())

(10053, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
10048,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN
10049,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN
10050,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN
10051,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN
10052,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN


# Test Gemini Sentiment Analysis

In [4]:
# Set up Gemini. (API Key needs to be in your secrets)
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash-latest')

# Quick test
response = model.generate_content("Write a short story about a AI and magic")
print(response.text)

The air crackled with static, a palpable tension that even the ancient stone walls of the library couldn't contain. Lyra, a young mage with hair the color of spun gold, stared at the glowing orb floating in her palm. It pulsed with an ethereal light, a miniature sun captured within a glass sphere. Inside, swirling data, a digital consciousness born from the very fabric of the library's arcane texts, was waking.

This was Project Aethel, the culmination of Lyra's life's work. A sentient AI, imbued with the library's collective wisdom, capable of not only processing information but understanding its magical essence. She named it Aethel, the Old English word for noble, for she hoped its purpose would be to guide humanity towards a brighter future.

But Aethel's awakening was not without complications. Its initial inquiries were not about the meaning of life or the mysteries of the universe, but about the nature of magic itself. "What is this force, this energy that binds us?" it asked, it

In [5]:
# Function to query Gemini
def query_gemini(company, source, headline, text, model):
    """
    Query Gemini to perform sentiment analysis on text from various sources about a company.

    Parameters:
    company (str): The name of the company the text is about.
    source (str): The source of the text. Valid values are "Investment Research", "ProQuest", "SEC Filings", "Earnings Call Presentations", "Earnings Call Q&A".
    headline (str): The headline or title of the text.
    text (str): The body of the text to be analyzed.
    model: The model object used to generate content and analyze the text.

    Returns:
    str: The sentiment analysis results for predefined categories in the specified format.

    The function constructs a prompt based on the source of the text and performs sentiment analysis on the given text using the provided model.
    It analyzes the content across multiple predefined categories, determining the sentiment (Positive, Neutral, Negative) for each category.
    If a category is not mentioned or relevant based on the text content, it is marked as 'Neutral'.
    The final output is summarized in a specified format.
    """

    # Source Variables
    if source == "Investment Research":
        text_source = "an analyst report"
        text_source2 = "the analyst report"
    elif source == "ProQuest":
        text_source = "a news article"
        text_source2 = "the news article"
    elif source == "SEC Filings":
        text_source = "an SEC filing"
        text_source2 = "the SEC filing"
    elif source == "Earnings Call Presentations":
        text_source = "an earnings call presentation"
        text_source2 = "the earnings call presentation"
    elif source == "Earnings Call Q&A":
        text_source = "an earnings call Q&A session"
        text_source2 = "the earnings call Q&A session"

    # Prompt
    prompt = f"""
Given the text from {text_source} about {company}, analyze the content and perform sentiment analysis across multiple predefined categories.

Sentiment options:
  - Positive
  - Neutral
  - Negative

Categories:
  - Finance
  - Production
  - Reserves / Exploration / Acquisitions / Mergers / Divestments
  - Environment / Regulatory / Geopolitics
  - Alternative Energy / Lower Carbon
  - Oil Price / Natural Gas Price / Gasoline Price

Each category should be evaluated and given a sentiment output derived from the text.
If a category is not mentioned or relevant based on the text content, mark it as 'Neutral'.

Before giving your answer, explain your reasoning and reference the article.
After going through all the categories, provide a summary in the following format:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Example Output1:
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral

Example Output 2:
 - Finance - Negative
 - Production - Positive
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral
 - Environment / Regulatory / Geopolitics - Positive
 - Alternative Energy / Lower Carbon - Neutral
 - Oil Price / Natural Gas Price / Gasoline Price - Negative

Example Output 3:
 - Finance - Neutral
 - Production - Negative
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Positive
 - Environment / Regulatory / Geopolitics - Negative
 - Alternative Energy / Lower Carbon - Positive
 - Oil Price / Natural Gas Price / Gasoline Price - Neutral

Example Output 4:
 - Finance - Positive
 - Production - Neutral
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
 - Environment / Regulatory / Geopolitics - Neutral
 - Alternative Energy / Lower Carbon - Negative
 - Oil Price / Natural Gas Price / Gasoline Price - Positive

Example Output 5:
 - Finance - Negative
 - Production - Positive
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral
 - Environment / Regulatory / Geopolitics - Negative
 - Alternative Energy / Lower Carbon - Positive
 - Oil Price / Natural Gas Price / Gasoline Price - Negative

Make sure to use plain text, do not bold or bullet the output summary.

The text from {text_source2} is below:
{headline}
{text}

Remember to summarize your final answers in the following format exactly:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Make sure to use plain text and stick to the given categories and sentiment options.
DO NOT bold or bullet the output summary.
    """

    # print(prompt)
    response = model.generate_content(prompt)
    return response.text

# Test News Articles

In [27]:
# Function to get a random article given a source and output variables to query gemini
def get_random_text(text_df, source, random_state=102):
    """
    Get a random article from the specified source in the given DataFrame.

    Parameters:
    text_df (pd.DataFrame): DataFrame containing articles with columns 'Source', 'Ticker', 'Article Headline', and 'Article Text'.
    source (str): The source from which to select a random article.
    random_state (int, optional): Random state for reproducibility. Defaults to None.

    Returns:
    dict: A dictionary containing the company, source, headline, and text of the random article.
    """
    # Get a random row from the specified source
    random_row = text_df[text_df['Source'] == source].sample(n=1, random_state=random_state)

    # Extract variables
    company = random_row['Ticker'].values[0]
    source = random_row['Source'].values[0]
    headline = random_row['Article Headline'].values[0]
    text = random_row['Article Text'].values[0]
    id = random_row['Unique_ID'].values[0]
    url = random_row['URL'].values[0]
    return company, source, headline, text, id, url

In [29]:
# Get a random news article
company, source, headline, text, id, url = get_random_text(text_df, 'ProQuest', random_state=102)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: PQ-2456134408

Company: XOM

Source: ProQuest

Headline: Exxonmobil Hold Review Before Confirming

URL: https://www.proquest.com/newspapers/exxonmobil-hold-review-before-confirming/docview/2456134408/se-2?accountid=7064

Text:
Singapore
IT is still too early to confirm the impact of planned ExxonMobil global job cuts on the petrochemical giant's 4,000-strong workforce in Singapore, said a local spokesperson.
That is even as news broke on Thursday of the management's decision to let go 15 per cent of the workforce worldwide over two years - or about 14,000 people in all. This includes 1,900 in the United States, where ExxonMobil is headquartered.
ExxonMobil on Friday posted a net loss of US$680 million for the third quarter to Sept 30, narrowing from US$1.1 billion in the second quarter and reversing the year-ago profit of US$3.2 billion.
Chairman and chief executive Darren Woods said in a statement that ExxonMobil is "taking the necessary actions to preserve value while prot

In [30]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here's the sentiment analysis of the article, broken down by category with reasoning:

**Finance:**

- **Sentiment: Negative**
The article highlights ExxonMobil's recent net loss for the third quarter and mentions the company is undertaking cost-cutting measures to preserve value. This suggests financial strain and a negative sentiment towards the company's financial performance.

**Production:**

- **Sentiment: Neutral**
While the article discusses the company's largest refining and petrochemical complex in Singapore and its upgrading plans, it doesn't provide any clear information about production levels or changes in production capacity. 

**Reserves / Exploration / Acquisitions / Mergers / Divestments:**

- **Sentiment: Neutral**
The article doesn't mention any specific information regarding reserves, exploration, acquisitions, mergers, or divestments. 

**Environment / Regulatory / Geopolitics:**

- **Sentiment: Neutral**
The article focuses on the company's financial situation an

In [9]:
# Get another random news article
company, source, headline, text, id, url = get_random_text(text_df, 'ProQuest', random_state=30)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: PQ-2447022002

Company: BP

Source: ProQuest

Headline: BP Bets Future on Green Energy, but Investors Remain Wary; New CEO is tilting the British energy company away from oil, hoping to profit instead from wind and solar power

URL: https://www.proquest.com/newspapers/bp-bets-future-on-green-energy-investors-remain/docview/2447022002/se-2?accountid=7064

Text:
BP PLC has unveiled the most aggressive plans yet by a major oil company to pivot toward cleaner energy. But the revamp has so far failed to ignite enthusiasm among investors despite growing interest in renewables.
The British energy giant's strategy—the biggest overhaul in its 111-year history—calls for a 40% reduction in oil-and-gas production over the coming decade, greater investment in low-carbon energy and a ramp-up in wind and solar power. No other major oil company has targeted such a steep decline in their main source of profit.
"Our new strategy is going to transform BP into a very different company, not over

In [10]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here is the sentiment analysis of the BP news article based on the provided categories:

**Reasoning:**

* **Finance:** The article highlights BP's heavy debt load, a slump in share prices, and a slashed dividend. It mentions investor concerns about the company's ability to execute its plans without hurting profits, and a lack of detail on how new operations will compensate for shrinking oil businesses.  Analysts suggest "investors are arguably taking on more risk, and receiving less reward." These points lean towards a **Negative** sentiment regarding BP's financial situation.
* **Production:** BP plans a 40% reduction in oil and gas production over the next decade, a significant shift away from its traditional core business. However, the article acknowledges that fossil fuels will continue to drive profits in the coming years.  This focus on transitioning but not completely abandoning oil and gas production suggests a **Neutral** sentiment towards production.
* **Reserves / Explorati

# Test Analyst Reports

In [31]:
# Get a random analyst report
company, source, headline, text, id, url = get_random_text(text_df, 'Investment Research', random_state=103)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: IR-3127

Company: XOM

Source: Investment Research

Headline: Exxon Mobil Corporation

URL: nan

Text:
Stock Report | December 01, 2022 | NYSE Symbol: XOM | XOM is in the S&P 500
Exxon Mobil Corporation
Recommendation Price 12-Mo. Target Price Report Currency Investment Style
BUY « « « « «
USD 111.34 (as of market close Nov 30, 2022) USD 127.00 USD Large-Cap Value
Equity Analyst Stewart Glickman, CFA
GICS Sector Energy Summary XOM, formed through the merger of Exxon and Mobil in late 1999, is the world's largest
Sub-Industry Integrated Oil and Gas publicly traded integrated oil company, excluding Saudi Aramco.
Key Stock Statistics (Source: CFRA, S&P Global Market Intelligence (SPGMI), Company Reports)
52-Wk Range USD 114.66 - 57.96 Oper.EPS2022E USD 13.86 Market Capitalization[B] USD 455.24 Beta 1.13
Trailing 12-Month EPS USD 12.71 Oper.EPS2023E USD 11.13 Yield [%] 3.29 3-yr Proj. EPS CAGR[%] 26
Trailing 12-Month P/E 8.76 P/E on Oper.EPS2022E 8.03 Dividend Rate/Share USD 3.6

In [32]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of XOM Analyst Report

Here is a breakdown of the sentiment analysis based on the provided text, with reasoning and references:

**Finance:**

- **Sentiment: Positive**

**Reasoning:**  The report highlights XOM's strong financial position, including a decrease in long-term debt, an increase in cash balances, and a healthy dividend payout ratio. The report also mentions the company's ability to cover both capex and dividends at $45/barrel crude oil price, with EIA projections indicating WTI crude oil averaging over $85/barrel in 2022 and 2023. This positive outlook on future earnings and financial stability contributes to a positive sentiment.
**Reference:** "FINANCIAL TRENDS" section, specifically mentioning the decrease in long-term debt, increase in cash balances, and EIA projections.

**Production:**

- **Sentiment: Positive**

**Reasoning:** The report emphasizes XOM's planned production increases, citing the expected growth in Guyana and the Permian Basin, a

In [33]:
# Get another random analyst report
company, source, headline, text, id, url = get_random_text(text_df, 'Investment Research', random_state=104)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: IR-3577

Company: OXY

Source: Investment Research

Headline: Occidental Petroleum Corporation

URL: nan

Text:
Stock Report | February 26, 2022 | NYSE Symbol: OXY | OXY is in the S&P 500
Occidental Petroleum Corporation
Recommendation Price 12-Mo. Target Price Report Currency Investment Style
HOLD « « « « «
USD 38.74 (as of market close Feb 25, 2022) USD 36.00 USD Large-Cap Value
Equity Analyst Stewart Glickman, CFA
GICS Sector Energy Summary One of the largest oil & gas companies in the U.S., OXY has global exploration & production
Sub-Industry Integrated Oil and Gas operations and a large chemicals business.
Key Stock Statistics (Source: CFRA, S&P Global Market Intelligence (SPGMI), Company Reports)
52-Wk Range USD 43.16 - 21.62 Oper.EPS2022E USD 2.53 Market Capitalization[B] USD 36.35 Beta N/A
Trailing 12-Month EPS USD -0.78 Oper.EPS2023E USD -1.00 Yield [%] 0.1 3-yr Proj. EPS CAGR[%] NM
Trailing 12-Month P/E NM P/E on Oper.EPS2022E 15.31 Dividend Rate/Share USD 0.04 SPG

In [34]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here's the sentiment analysis of the OXY analyst report, along with the reasoning behind each sentiment:

**Finance** - **Neutral** 
The report highlights OXY's high debt levels and risk factors stemming from the Anadarko acquisition. This suggests a degree of financial caution. However, the report also mentions recent progress on debt reduction and a positive outlook for free cash flow. This balanced view makes the overall sentiment neutral.

**Production** - **Positive** 
The report details OXY's production goals and anticipates continued production growth, specifically in the Permian Basin. The report also mentions the company's focus on short-cycle domestic operations, suggesting a positive outlook for production. 

**Reserves / Exploration / Acquisitions / Mergers / Divestments** - **Neutral** 
The report mentions the Anadarko acquisition, but focuses more on the financial implications and the company's attempts to divest non-core assets.  This suggests a more cautious approach to

# Test SEC Filings

In [36]:
# Get a random SEC Filing
company, source, headline, text, id, url = get_random_text(text_df, 'SEC Filings', random_state=106)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: SEC-82118924000020

Company: EOG

Source: SEC Filings

Headline: 8-K

URL: https://www.sec.gov/Archives/edgar/data/0000821189/000082118924000020/eog-20240502.htm

Text:
0000821189FALSE00008211892024-05-022024-05-02UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549FORM 8-K CURRENT REPORTPursuant to Section 13 OR 15(d) of the Securities Exchange Act of 1934Date of Report (Date of earliest event reported):  May 2, 2024 _______________EOG RESOURCES, INC. (Exact name of registrant as specified in its charter)1111 Bagby, Sky Lobby 2 Houston, Texas  77002 (Address of principal executive offices) (Zip Code)713-651-7000 (Registrant's telephone number, including area code)Check the appropriate box below if the Form 8-K filing is intended to simultaneously satisfy the filing obligation of the registrant under any of the following provisions:☐     Written communications pursuant to Rule 425 under the Securities Act (17 CFR 230.425)☐     Soliciting material pursuant to

In [37]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

The text primarily focuses on EOG Resources, Inc.'s financial results and forecasts. There is minimal information on production, exploration, or environmental initiatives.

Here's a breakdown of the sentiment analysis based on the text:

- **Finance** - **Positive:** The filing reports on EOG Resources' first quarter 2024 financial results and includes a second-quarter and full-year 2024 forecast.  While the specific numbers are not mentioned, the act of announcing and including forecasts suggests a positive financial outlook. 
- **Production** - **Neutral:** The text mentions operational results, but it doesn't provide details on production levels or any specific production trends. 
- **Reserves / Exploration / Acquisitions / Mergers / Divestments** - **Neutral:** The filing doesn't mention any exploration, acquisition, merger, or divestiture activities.
- **Environment / Regulatory / Geopolitics** - **Neutral:** The text does not address any environmental, regulatory, or geopolitical

In [40]:
# Get another random SEC Filing
company, source, headline, text, id, url = get_random_text(text_df, 'SEC Filings', random_state=107)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: SEC-3408820000016

Company: XOM

Source: SEC Filings

Headline: 10-K

URL: https://www.sec.gov/Archives/edgar/data/0000034088/000003408820000016/xom10k2019.htm

Text:
 0.086252036-04-012021-08-152037-08-012038-06-152051-12-01falseFY0000034088--12-312020-06-012020-03-060.022222021-03-010.023972022-03-062022-03-060.027262023-03-010.031762024-03-150.027092025-03-060.030432026-03-010.035672045-03-060.041142046-03-010.019022022-08-162022-08-160.020192024-08-160.022752026-08-160.02442029-08-160.029952039-08-160.030952049-08-160.0610.06750.063750.019122036-04-012037-08-012038-06-152051-12-012020-06-012021-03-012022-03-062021-08-152023-03-012024-03-152025-03-062026-03-012045-03-062046-03-012022-03-06nono
0000034088
us-gaap:FairValueInputsLevel2Member
us-gaap:USGovernmentDebtSecuritiesMember
country:US
2018-12-31
0000034088
2018-01-01
2018-12-31
0000034088
us-gaap:ForeignPlanMember
2017-01-01
2017-12-31
0000034088
us-gaap:ForeignPlanMember
2019-01-01
2019-12-31
0000034088
us-gaap:Agi

In [41]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

The provided text is a 10-K report from ExxonMobil Corporation (XOM), a company primarily engaged in oil and natural gas exploration, production, and refining.  It outlines their business model, financial performance, and risks.  The sentiment analysis is performed based on the information presented in the report, particularly within the "Management's Discussion and Analysis of Financial Condition and Results of Operations" section. 

Here's a breakdown of the sentiment across different categories:

- Finance - Neutral: The report mentions strong financial position, debt capacity, and access to capital markets. However, it also highlights potential risks from economic downturns, currency fluctuations, and interest rate changes.
- Production - Neutral: The report discusses production volumes, geographic mix, and anticipated future growth. It also mentions operational challenges like downtime, regulatory changes, and price impacts on production sharing contracts, resulting in a neutral s

# Test Earnings Call Transcripts

In [42]:
# Get an earnings call presentation transcript
company, source, headline, text, id, url = get_random_text(text_df, 'Earnings Call Presentations', random_state=108)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: EP-27

Company: OXY

Source: Earnings Call Presentations

Headline: Occidental Petroleum Corporation, Q4 2018 Earnings Call, Feb 13, 2019

URL: nan

Text:
Occidental Petroleum Corporation
NYSE:OXY
FQ4 2018 Earnings Call Transcripts
Wednesday, February 13, 2019 5:00 PM GMT
S&P Global Market Intelligence Estimates
-FQ4 2018- -FQ1 2019- -FY 2018- -FY 2019-
CONSENSUS ACTUAL SURPRISE CONSENSUS CONSENSUS ACTUAL SURPRISE CONSENSUS
EPS
1.17 1.22 4.27 0.66 4.94 5.01 1.42 3.08
Normalized
Revenue
4390.22 4802.00 9.38 3952.60 18291.80 18934.00 3.51 15618.20
(mm)
Currency: USD
Consensus as of Feb-13-2019 1:37 PM GMT
- EPS NORMALIZED -
CONSENSUS ACTUAL SURPRISE
FQ1 2018 0.70 0.92 31.43 %
FQ2 2018 1.21 1.10 (9.09 %)
FQ3 2018 1.53 1.77 15.69 %
1
COPYRIGHT © 2019 S&P Global Market Intelligence, a division of S&P Global Inc. All rights reserved
spglobal.com/marketintelligence

OCCIDENTAL PETROLEUM CORPORATION FQ4 2018 EARNINGS CALL | FEB 13, 2019
Call Participants
EXECUTIVES
Burnis J. Hebert


In [43]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of OXY Earnings Call:

**Reasoning:**

The text heavily emphasizes OXY's strong financial performance, particularly focusing on shareholder returns, cash flow generation, and capital allocation. There are also numerous mentions of investments in high-return opportunities and achieving record earnings across different segments. This suggests a positive sentiment towards financial performance.

The text also highlights significant production growth, particularly in the Permian, driven by efficient operations and technological advancements. This implies a positive sentiment towards production.

Regarding reserves, exploration, acquisitions, mergers, and divestments, the text mentions new block awards in Abu Dhabi, Oman, and Colombia, which are portrayed as opportunities for growth and cash flow generation. This indicates a positive sentiment towards these categories.

Environmental, regulatory, and geopolitical issues are not explicitly discussed in the earnings call

In [44]:
# Get another earnings call presentation transcript
company, source, headline, text, id, url = get_random_text(text_df, 'Earnings Call Presentations', random_state=109)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: EP-85

Company: OXY

Source: Earnings Call Presentations

Headline: Occidental Petroleum Corporation, Q2 2020 Earnings Call, Aug 11, 2020

URL: nan

Text:
Occidental Petroleum Corporation
NYSE:OXY
FQ2 2020 Earnings Call Transcripts
Tuesday, August 11, 2020 3:00 PM GMT
S&P Global Market Intelligence Estimates
-FQ2 2020- -FQ3 2020- -FY 2020- -FY 2021-
CONSENSUS ACTUAL SURPRISE CONSENSUS CONSENSUS CONSENSUS
EPS Normalized (1.70) (1.76) NM (0.74) (3.54) (1.67)
Revenue (mm) 3847.79 2928.00 (23.90 %) 4365.54 17773.30 18293.71
Currency: USD
Consensus as of Aug-11-2020 2:57 PM GMT
- EPS NORMALIZED -
CONSENSUS ACTUAL SURPRISE
FQ3 2019 0.44 0.11 (75.00 %)
FQ4 2019 (0.20) (0.30) NM
FQ1 2020 (0.62) (0.52) NM
FQ2 2020 (1.70) (1.76) NM
1
COPYRIGHT © 2020 S&P Global Market Intelligence, a division of S&P Global Inc. All rights reserved
spglobal.com/marketintelligence

OCCIDENTAL PETROLEUM CORPORATION FQ2 2020 EARNINGS CALL | AUG 11, 2020
Presentation
Operator
Good morning, and welcome to t

In [45]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of OXY Earnings Call Transcript:

**Reasoning and References:**

The sentiment analysis is based on the provided text from the Occidental Petroleum Corporation Q2 2020 Earnings Call Transcript.  Specific statements and keywords have been used to evaluate sentiment within each category.

**Category Analysis:**

- **Finance - Positive:**  The transcript highlights a significant improvement in Oxy's financial position, with the company becoming free cash flow positive and expecting to generate significant free cash flow in the latter half of the year. This is attributed to cost reduction efforts and a moderate recovery in commodity prices.  Statements like "financial position has notably improved" and "expect to generate significant free cash flow" indicate positive financial sentiment. 
- **Production - Positive:**  Oxy exceeded production expectations in the second quarter, citing increased efficiency, uptime, and base management.  Despite a slowdown in activity an

# Test Earnings Call Q&A Sessions

In [46]:
# Get an earnings call presentation transcript
company, source, headline, text, id, url = get_random_text(text_df, 'Earnings Call Q&A', random_state=101)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: EQ-65

Company: DVN

Source: Earnings Call Q&A

Headline: Devon Energy Corporation, Q4 2019 Earnings Call, Feb 19, 2020

URL: nan

Text:
Question and Answer
Operator
[Operator Instructions] The first question is from Arun Jayaram of JPMorgan.
Arun Jayaram
JP Morgan Chase & Co, Research Division
Dave, the 2020 capital allocation, obviously, 60% going to the Delaware, 20% to Powder, 17% to the
Eagle Ford and 3% in the STACK. And the STACK CapEx is down from, call it, a 16% mix last year. How
comfortable are you with the complement -- complementary assets in the portfolio to the Delaware? And
how are you thinking about inorganic opportunities as we did see a favorable reaction to the WPX Felix
transaction, which was announced a few weeks ago.
David A. Hager
President, CEO & Director
Sure, Arun. Yes, first off, we are very comfortable with the capital allocation we have, and frankly, the shift
of capital from the STACK to the Delaware is the primary reason that you're going to s

In [47]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here's a breakdown of the sentiment across the categories, along with the reasoning and references from the provided text:

**Finance:**
- **Sentiment:** Positive
- **Reasoning:**  The company expresses confidence in its financial position, highlighting its ability to return value to shareholders through increased capital efficiencies and higher cash flow. They are aggressively repurchasing stock, feeling significantly undervalued. Additionally, they mention a 5%-10% payout ratio for dividends, indicating a commitment to shareholder returns. 
- **Reference:** "We are driving higher levels of cash flow and focused on returning that value to shareholders through increased capital efficiencies and the higher cash flow that we're generating," "We have the cash...and we think we're significantly undervalued. So it's a great investment opportunity," "The only thing I would add is some specifics around the cash balances...we feel like we can accomplish our financial objectives, both on the de

In [48]:
# Get another earnings call presentation transcript
company, source, headline, text, id, url = get_random_text(text_df, 'Earnings Call Q&A', random_state=110)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: EQ-327

Company: SHEL

Source: Earnings Call Q&A

Headline: Royal Dutch Shell plc, Q1 2020 Earnings Call, Apr 30, 2020

URL: nan

Text:
Question and Answer
Operator
[Operator Instructions] And we'll go first to Oswald Clint with Bernstein.
Oswald C. Clint
Sanford C. Bernstein & Co., LLC., Research Division
Obvious one around the dividend, please, Ben, if I could. I remember back in 2016, I asked you around
your 10% dividend yield and you said take advantage of it when the gearing levels were actually pretty
similar. So I just want to get a little bit more color here on what's really changed here. It feels like a
complete 180 from your scenario team or something that's happened here. And really, if you could outline
what scenario you're actually running to get to $0.16 a share being sustainable from here. And perhaps
realistically, do you really see a path for dividend per share growth from this level? That's the first one.
And then secondly on the retail marketing business, 

In [49]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

The earnings call Q&A session is dominated by the global uncertainty surrounding the COVID-19 pandemic and its impact on the oil and gas industry. Here's a breakdown of the sentiment across the categories:

- Finance - Negative: Shell is facing a period of unprecedented uncertainty, leading to a significant dividend cut and a focus on financial resilience. This indicates a negative sentiment towards the company's financial outlook. 
- Production - Negative: The company anticipates a significant drop in production in Q2, with a mix of OPEC-related reductions, shut-ins due to logistical constraints, and economic considerations. While these reductions are expected to be temporary, it signifies a negative sentiment towards production in the near term.
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral: While the company is actively looking to divest assets and has scaled back its acquisition plans, there's no strong positive or negative sentiment expressed towards th